In [73]:
# Treasury Forecasting algo version #2 determining the best investnent windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=False)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [74]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [75]:
# STEP 1: Get the asset classes combined with their parent class ID
table_name = 'AssetClass'
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
asset_classes = fetch_data(table_name, '',1,sql)
asset_classes

,ID,Title,Group,Issuer,PercentMax,AssetClassCombined
0,1,Cash/Sweep,Not Assigned,None,1.0000,Cash/Sweep
1,2,Certificate of Deposit,Certificates of Deposit,None,0.5000,Certificate of Deposit
2,3,Commercial Paper,Commercial Paper,None,0.3330,Commercial Paper
3,4,US Agencies,US Agencies,None,0.2000,US Agencies
4,7,Money Market,Not Assigned,None,0.2000,Money Market
5,8,Mutual Fund,Not Assigned,None,0.2000,Mutual Fund
6,16,US Treasuries,US Treasuries,None,1.0000,US Treasuries


In [76]:
# STEP 4: Running balance Day view taken from the SQL views
#           Q: Do we want to replace the SQL views with panasda dataframes?

table_name = 'RunningBalanceDayView'
running_balances = fetch_data(table_name)
running_balances


,TransactionDate,TransactionClass,RunningTotal
0,2025-01-21,Portfolio,654628445.37
1,2025-01-22,Portfolio,492520445.37
2,2025-01-23,Portfolio,492520445.37
3,2025-01-24,Portfolio,470088445.37
4,2025-01-25,Portfolio,470088445.37
...,...,...,...
4971,2026-09-30,US Treasuries,-1883266.58
4972,2026-10-01,US Treasuries,-1883266.58
4973,2026-10-02,US Treasuries,-1883266.58
4974,2026-10-03,US Treasuries,-1883266.58


In [77]:
# Add the daily total portfolio balance to the running balances DataFrame

# Convert TransactionDate to datetime if not already
running_balances['TransactionDate'] = pd.to_datetime(running_balances['TransactionDate'])

# Filter and pivot Portfolio balances
portfolio_balances = running_balances[running_balances['TransactionClass'] == 'Portfolio'].copy()
portfolio_balances['RunningTotal'] = pd.to_numeric(portfolio_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_portfolio = portfolio_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_portfolio.reset_index().rename(columns={'RunningTotal': 'Portfolio'}),
    on='TransactionDate',
    how='left'
)

In [78]:
# Add the asset class PercentMax column to the running balances DataFrame matching the on TransactionClass colunmn
# Convert PercentMax to float and merge with running_balances
asset_classes['PercentMax'] = pd.to_numeric(asset_classes['PercentMax'])
running_balances = running_balances.merge(
    asset_classes[['Title', 'PercentMax']],
    left_on='TransactionClass',
    right_on='Title',
    how='left'
).drop('Title', axis=1).fillna(1) # Set default to 100% if not specified


In [79]:
# Compute the asset class's maximum based on the policy
running_balances['PolicyMax'] = running_balances['Portfolio'] * running_balances['PercentMax']


In [80]:
# Add the daily available cash balance to the running balances DataFrame

# Filter and pivot cash balances
cash_balances = running_balances[running_balances['TransactionClass'] == 'Cash/Sweep'].copy()
cash_balances['RunningTotal'] = pd.to_numeric(cash_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_cash = cash_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_cash.reset_index().rename(columns={'RunningTotal': 'CashSweep'}),
    on='TransactionDate',
    how='left'
)

In [81]:
# Add the amount investable column
#       if [TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then [PolicyMax]-[RunningTotal] else [RunningTotal]

# Convert RunningTotal to numeric if it's not already
running_balances['RunningTotal'] = pd.to_numeric(running_balances['RunningTotal'])

# Add Investable column based on the condition
running_balances['Investable'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') &
    (running_balances['TransactionClass'] != 'Cash/Sweep'),
    running_balances['PolicyMax'] - running_balances['RunningTotal'],
    running_balances['RunningTotal']
)


In [82]:
# Add the final available column
#       if ([TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then MIN([CashSweep],[Investable]) else [RunningTotal])
running_balances['Available'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') &
    (running_balances['TransactionClass'] != 'Cash/Sweep'),
    np.minimum(running_balances['CashSweep'], running_balances['Investable']),
    running_balances['RunningTotal']
)

In [83]:
# Output the final datafram

# Suppress scientific notation by setting float_format
pd.options.display.float_format = '{:,.0f}'.format

# Display the dataframe without scientific notation
print(running_balances[running_balances['TransactionClass'] == 'Money Market'])

     TransactionDate TransactionClass  RunningTotal   Portfolio  PercentMax  \
2488      2025-01-21     Money Market    20,065,581 654,628,445           0   
2489      2025-01-22     Money Market    20,065,581 492,520,445           0   
2490      2025-01-23     Money Market    20,065,581 492,520,445           0   
2491      2025-01-24     Money Market    20,065,581 470,088,445           0   
2492      2025-01-25     Money Market    20,065,581 470,088,445           0   
...              ...              ...           ...         ...         ...   
3105      2026-09-30     Money Market    20,065,581 412,783,445           0   
3106      2026-10-01     Money Market    20,065,581 412,783,445           0   
3107      2026-10-02     Money Market    20,065,581 412,783,445           0   
3108      2026-10-03     Money Market    20,065,581 412,783,445           0   
3109      2026-10-04     Money Market    20,065,581 412,783,445           0   

       PolicyMax   CashSweep  Investable   Availabl